In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(
    "D:/Projects/Lattice interview 2/Summarise Report/",
    glob="*.txt",
    loader_cls=TextLoader
)
docs = loader.load()


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = FAISS.from_documents(chunks, embeddings) #Qdrant is a better option for production use


In [ ]:

query = "was there any images of the site taken"
relevant_docs = db.similarity_search(query)
print(relevant_docs)
context = "\n\n".join([doc.page_content for doc in relevant_docs])
prompt = f"""You are going to Answer the question based on the context given below, be consize and try direct and correct answers only:\n\n \n{context}\n\nQuestion: {query}<|im_end|>
<|im_start|> assistant"""
print(context)


[Document(id='99364655-ee14-4320-9208-2526f3385292', metadata={'source': 'D:\\Projects\\Lattice interview 2\\Summarise Report\\2025-06-19_ground_site_report.txt'}, page_content="Daily Site Report - 2025-06-19\n\nToday's report highlights the following issues and tasks:\n\n1. Safety Notice (Green) - Good Observation** - Employee ID 524572 reported good, clear exclusion zones and access throughout the site, contributing to a safe working environment.\n\n2. Progress Photo** - Employee ID 524575 took progress photos of the ongoing RC walls construction, showing steady advancement in the structural phase."), Document(id='ecdef388-78fa-45e4-a2af-8c772f7f9a93', metadata={'source': 'D:\\Projects\\Lattice interview 2\\Summarise Report\\2025-06-20_ground_site_report.txt'}, page_content='Daily Site Report - 2025-06-22\n\nNo work done, Bad weather holiday'), Document(id='24c5678a-7357-471d-b173-f1a4c813b15d', metadata={'source': 'D:\\Projects\\Lattice interview 2\\Summarise Report\\2025-06-21_grou

In [30]:
import requests
LLAMA_URL = "http://127.0.0.1:8080/completion"
rompt = f"""
<|im_start|> You: Answer the question based on the context given below, be concise and try direct and correct answers only:\n\n{context}\n\nQuestion: {query}
Observations:
<|im_end|>
<|im_start|> assistant
"""
payload = {
                "prompt": prompt,
                "temperature": 0.7, 
                "n_predict": 512,  # for longer reports
                "stream": False,
                "stop": ["<|im_end|>"],
            }
response = requests.post(LLAMA_URL, json=payload, timeout=120) # Increased timeout
response.raise_for_status()
result = response.json()
ai_generated_report = result.get("content", "").strip()
print(ai_generated_report)

: Yes, a progress photo was taken on the 19th of June by Employee ID 524575.
